# ~ L200921 (Aisha Muhammad Nawaz) ~
## ~ CS4080 Mining Massive Datasets (BSCS 8A Spring 2024) ~
## ~ Assignment 1 - Map Reduce Basics (Due Date: 12 Feb 2024) ~
SUBMISSION: Upload the Source code and the output file on Google Classroom in a zip file with your roll number.

INPUT FILE: You are given an input text file named citation.txt. It contains information regarding the research papers published in various journals. The complete file Citation-network V1 can be found at https://cn.aminer.org/citation. The format of the file is as follows:

#* --- paperTitle
#@ --- Authors
#t ---- Year
#c --- publication venue
#index 00---- index id of this paper

QUESTION: Write an efficient MapReduce program for the following problems. To make your algorithm efficient, you should use combiners or in-mapper aggregation techniques that use arrays.
## REVISTED SOLUTIONS ONLY
#### Note: GC File is the file uploaded on Google Classroom assignment post

### 4. Find the average number of papers published each year.

In [19]:
%%file q4.py
#*NOTE: I am assuming the average number of papers published each year means Total papers published / Total Distinct Years.
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class AvgPapersPublishedEachYear(MRJob):
    def configure_args(self):
        super(AvgPapersPublishedEachYear, self).configure_args()
        self.add_file_arg('--filename', help='Path to the input file') #TO make sure file opens only once
    def init_read_file(self):
        self.patternYear=re.compile(r"(#t[^\#]*)")
        self.papersPerYearSum={}
    def get_papers_count(self,_,line):
        file=[file for file in line.split('#*') if len(file)>0]
        if file:
            year=''.join(self.patternYear.findall(file[0])).replace('#t','').replace('\n','').replace(' ','')
            if(year):
                year=int(year)
                self.papersPerYearSum.setdefault(year,0)
                self.papersPerYearSum[year]=self.papersPerYearSum[year]+1
            
    def final_get_papers_count(self):
        for year,value in self.papersPerYearSum.items():
            yield year,value
            
    def sum_years_count(self,year,value):
        yield year,sum(value)
        
    def sum_years_count_red(self,year,value):
        yield None,(year,sum(value))
        
    def avg_years_count(self,_,value):
        sumValue=0
        sumCount=0
        for count,val in value:
            sumValue=sumValue+val
            sumCount=sumCount+1
        yield "Average Papers Published Each Year = ",(sumValue/sumCount)

    def steps(self):
        return [
            MRStep(mapper_init=self.init_read_file,
                  mapper=self.get_papers_count,
                  mapper_final=self.final_get_papers_count,
                  combiner=self.sum_years_count,
                  reducer=self.sum_years_count_red
                  ),
            MRStep(
                  reducer=self.avg_years_count
                  )
        ]
        
if __name__=='__main__':
    AvgPapersPublishedEachYear.run()

Overwriting q4.py


### Q4 GC FILE OUTPUT

In [8]:
!python q4.py citation.txt

"Average Papers Published Each Year = "	4.545454545454546


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240212.172211.380475
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240212.172211.380475\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240212.172211.380475\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240212.172211.380475...


In [20]:
!python q4.py citation.txt

"Average Papers Published Each Year = "	4.545454545454546


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240602.052238.057217
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240602.052238.057217\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240602.052238.057217\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\q4.DELL.20240602.052238.057217...


### 6. Find the names of authors who have written at most one paper in a year.

In [23]:
%%file q6.py
#*NOTE: I am assuming here 'at most one paper in a year' means the author has some year in which they wrote one paper (This may or may not be their max in all years)
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

class CitationsAuthorsPerYearCountMaxOne(MRJob):
    def configure_args(self):
        super(CitationsAuthorsPerYearCountMaxOne, self).configure_args()
        self.add_file_arg('--filename', help='Path to the input file') #TO make sure file opens only once
    def init_read_file(self):
        self.patternAuthors=re.compile(r"\#\@[^\#\t]*")
        self.patternYear=re.compile(r"(#t[^\#]*)")
        self.authorsCount={}
    def get_authors_count(self,_,line):
        file=[file for file in line.split('#*') if len(file)>0]
        if (file):
            if("#@" in file[0]):
                year=''.join(self.patternYear.findall(file[0])).replace('#t','').replace('\n','').replace(' ','')
                if(year):
                    authors=''.join(self.patternAuthors.findall(file[0])).replace('#@','').replace('\n','')
                    authors=[isAuthor for isAuthor in authors.split(',') if len(isAuthor)>1]
                    for author in authors:
                        author=author.strip()
                        self.authorsCount.setdefault(author+'->'+year,0)
                        self.authorsCount[author+'->'+year]=self.authorsCount[author+'->'+year]+1

    def final_get_authors_count(self):
        for authorYear,count in self.authorsCount.items():
            yield authorYear,count
            
    def sum_authors_count_combiner(self,authorYear,count):
        yield authorYear,sum(count)
        
    def sum_authors_count_reducer(self,authorYear,count):
        valueSummed=sum(count)
        if(valueSummed==1):
            authorYear=authorYear.split('->')
            yield 'Author: '+authorYear[0],' Year: '+authorYear[1]            
            
    def steps(self):
        return [
            MRStep(
                  mapper_init=self.init_read_file,
                  mapper=self.get_authors_count,
                  mapper_final=self.final_get_authors_count,
                  combiner=self.sum_authors_count_combiner,
                  reducer=self.sum_authors_count_reducer)
        ]
        
if __name__=='__main__':
    CitationsAuthorsPerYearCountMaxOne.run()

Overwriting q6.py


### Q6 GC FILE OUTPUT

In [12]:
!python q6.py citation.txt

"Author: A. Krzyzak"	" Year: 1996"
"Author: Ahmed Hassan"	" Year: 2007"
"Author: Alessandro Aurigi"	" Year: 2005"
"Author: Alex Galis"	" Year: 2006"
"Author: Alexander Gelbukh"	" Year: 2006"
"Author: Alice Redmond-neal"	" Year: 2005"
"Author: Aline Maria Santos Andrade"	" Year: 2005"
"Author: Allan Hunkin"	" Year: 2007"
"Author: Amir Ahmad"	" Year: 2007"
"Author: Amitabh Chaudhary"	" Year: 2006"
"Author: Amitabha Bagchi"	" Year: 2006"
"Author: Andreas N\u00fcrnberger"	" Year: 2004"
"Author: Anita Kesavan"	" Year: 2006"
"Author: Ankur Bhargava"	" Year: 2006"
"Author: Arthur Greef"	" Year: 2006"
"Author: Arto Tapani Juhola"	" Year: 2006"
"Author: Axel Bucker"	" Year: 2005"
"Author: Barry Smyth"	" Year: 2006"
"Author: Bart Preneel"	" Year: 2006"
"Author: Behrouz A. Forouzan"	" Year: 2005"
"Author: Ben Long"	" Year: 2006"
"Author: Brenden Munnelly"	" Year: 2002"
"Author: Bruce Shriver"	" Year: 1975"
"Author: Carla Rose"	" Year: 1994"
"Author: Carlito Vicencio"	" Year: 2005"
"Author: Carlos

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240212.172215.925612
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240212.172215.925612\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240212.172215.925612\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240212.172215.925612...


In [24]:
!python q6.py citation.txt

"Author: A. Krzyzak"	" Year: 1996"
"Author: Ahmed Hassan"	" Year: 2007"
"Author: Alessandro Aurigi"	" Year: 2005"
"Author: Alex Galis"	" Year: 2006"
"Author: Alexander Gelbukh"	" Year: 2006"
"Author: Alice Redmond-neal"	" Year: 2005"
"Author: Aline Maria Santos Andrade"	" Year: 2005"
"Author: Allan Hunkin"	" Year: 2007"
"Author: Amir Ahmad"	" Year: 2007"
"Author: Amitabh Chaudhary"	" Year: 2006"
"Author: Amitabha Bagchi"	" Year: 2006"
"Author: Andreas N\u00fcrnberger"	" Year: 2004"
"Author: Anita Kesavan"	" Year: 2006"
"Author: Ankur Bhargava"	" Year: 2006"
"Author: Arthur Greef"	" Year: 2006"
"Author: Arto Tapani Juhola"	" Year: 2006"
"Author: Axel Bucker"	" Year: 2005"
"Author: Barry Smyth"	" Year: 2006"
"Author: Bart Preneel"	" Year: 2006"
"Author: Behrouz A. Forouzan"	" Year: 2005"
"Author: Ben Long"	" Year: 2006"
"Author: Brenden Munnelly"	" Year: 2002"
"Author: Bruce Shriver"	" Year: 1975"
"Author: Carla Rose"	" Year: 1994"
"Author: Carlito Vicencio"	" Year: 2005"
"Author: Carlos

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240602.052714.781107
Running step 1 of 1...
job output is in C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240602.052714.781107\output
Streaming final output from C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240602.052714.781107\output...
Removing temp directory C:\Users\DELL\AppData\Local\Temp\q6.DELL.20240602.052714.781107...
